### Binary Notebook

We start with the same 4 classes from the original notebook but convery the Tumor classes into 1 class, so now we only have Tumor vs Non Tumor

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2
import os
import shutil
import imghdr
#import optuna
#import keras_tuner as kt
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
IMAGE_SIZE = (256,256,3)
OPT = tf.keras.optimizers.legacy.Adam(learning_rate = 0.001)
BATCH_SIZE = 32
NUM_EPOCHS = 40

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
training_path = '/content/drive/MyDrive/projects/Brain_Tumor_Classification/Images_Brain/Training'
testing_path = '/content/drive/MyDrive/projects/Brain_Tumor_Classification/Images_Brain/Testing'

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
# Load the training dataset
Training_data = tf.keras.utils.image_dataset_from_directory(
    training_path,
    labels='inferred',
    image_size=(256, 256),
    batch_size=32,
    validation_split=0.2,
    subset='training',
    seed=42
)

# Load the validation dataset
Validation_data = tf.keras.utils.image_dataset_from_directory(
    training_path,
    labels='inferred',
    image_size=(256, 256),
    batch_size=32,
    validation_split=0.2,
    subset='validation',
    seed=42
)

# Load the testing dataset
Testing_data = tf.keras.utils.image_dataset_from_directory(
    testing_path,
    labels='inferred',
    image_size=(256, 256),
    batch_size=32,
)

In [ ]:
Training_iterator = Training_data.as_numpy_iterator()
Testing_iterator = Testing_data.as_numpy_iterator()


In [ ]:
batch_training = Training_iterator.next()

In [ ]:
batch_training[1]

In [ ]:
class_names = Training_data.class_names
print("Class Names:", class_names)

In [ ]:
fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
for idx, img_index in enumerate([12,13,14,15]):
    ax[idx].imshow(batch_training[0][img_index].astype(int))
    ax[idx].title.set_text(batch_training[1][img_index])


In [ ]:
# We are now dealing with just 2 lables, 0, 1, and 3 are tumor vs 2 which is non tumor



def augment_image(x, y):

    #  # Random left-right flip
    # x = tf.image.random_flip_left_right(x)
    # # Random up-down flip
    # x = tf.image.random_flip_up_down(x)

    #  # Random rotation (90-degree increments)
    # num_rotations = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    # x = tf.image.rot90(x, k=num_rotations)

    # # Random zoom (zooming in or out)
    # zoom_factor = tf.random.uniform(shape=[], minval=0.8, maxval=1.2)
    # crop_size = tf.cast(tf.cast(tf.shape(x)[0:2], dtype=tf.float32) * zoom_factor, dtype=tf.int32)
    # x = tf.image.central_crop(x, central_fraction=zoom_factor)
    # x = tf.image.resize(x, crop_size)
    # )
    # Add more augmentation functions as needed
    return x, y

def get_positions(x, y): # Gets the position of the 1 so we can identify the type of tumor and then create the label
    positions = tf.where(tf.equal(y, 1))
    positions = tf.gather(positions[:, 1], tf.where(y == 1)[:, 0])
    return x, positions

def map_labels(x, y):
    binary_labels = tf.where(tf.equal(y, 2), 0, 1)  # No tumor (2) becomes 0, others become 1
    return x, binary_labels



In [ ]:
tf.config.run_functions_eagerly(True)

num_classes = 4  # Adjust based on your number of classes


# Load the training dataset
Training_data = Training_data.map(augment_image)
Training_data = Training_data.map(lambda x, y: (x / 255.0, tf.one_hot(y, num_classes)))
Training_data = Training_data.map(get_positions)
Training_data = Training_data.map(map_labels)


# Load the validation dataset
Validation_data = Validation_data.map(augment_image)
Validation_data = Validation_data.map(lambda x, y: (x / 255.0, tf.one_hot(y, num_classes)))
Validation_data = Validation_data.map(get_positions)
Validation_data = Validation_data.map(map_labels)

# Load the testing dataset
Testing_data = Testing_data.map(augment_image)
Testing_data = Testing_data.map(lambda x, y: (x / 255.0, tf.one_hot(y, num_classes)))
Testing_data = Testing_data.map(get_positions)
Testing_data = Testing_data.map(map_labels)

# X is Images
# Y is Labels

In [ ]:
batch_training = Training_data.as_numpy_iterator().next()

In [ ]:
batch_training[1] # We can see the labels are now all 0 and 1 to represent Tumor vs Non Tumor, which is what we wanted

In [ ]:
fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
for idx, img_index in enumerate([1,2,7,17]):
    ax[idx].imshow(batch_training[0][img_index])
    ax[idx].title.set_text(batch_training[1][img_index])

In [ ]:
new_class_names = ['No Tumor', 'Tumor']
fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
for idx, img_index in enumerate([1,2,7,17]):
    ax[idx].imshow(batch_training[0][img_index])
    #ax[idx].title.set_text(batch_training[1][img_index])
    ax[idx].title.set_text(new_class_names[batch_training[1][img_index]])

## Deep Model


In [ ]:
model = Sequential()

# First have an input layer, going to have 16 filters, filter is a 3x3, stride of 1
# Relu activation turns negative values to 0, and preserves positive values
model.add(Conv2D(32, (3,3), 1, activation = 'relu', input_shape = IMAGE_SIZE))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Flatten()) # condense values

# Fully connected layers
model.add(Dense(64, activation = 'relu', kernel_regularizer=l2(0.002)))
# model.add(Dropout(rate=0.1))

model.add(Dense(256, activation = 'relu', kernel_regularizer=l2(0.02)))
#model.add(Dropout(rate=0.1))


# Final layer that gives a single output and represets the label
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer= OPT, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Training

In [ ]:
logdir = '/content/drive/MyDrive/projects/Brain_Tumor_Classification/logs'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir)

early_stopping = EarlyStopping(monitor='val_loss', patience=4)


In [ ]:
hist = model.fit(Training_data, epochs = 15, validation_data = Validation_data, callbacks = [tensorboard_callback, early_stopping])


## Evaluation and Testing

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()


In [ ]:
labels_testing = []
X_test =[]

for batch in Testing_data.as_numpy_iterator():
    x_test, y_test = batch
    labels_testing.extend(y_test)
    X_test.extend(x_test)

labels_testing = np.array(labels_testing)
X_test = np.array(X_test)



In [ ]:

y_ = labels_testing # Our true y values
yhat_ =[] # Empty list for predicted variables

yhat = model.predict(X_test)

yhat_binary = np.argmax(yhat, axis=1)

yhat_.append(yhat_binary)

pre.update_state(y_, yhat_binary)
re.update_state(y_, yhat_binary)
acc.update_state(y_, yhat_binary)

In [ ]:
print(f'Precision: {pre.result().numpy()}')
print(f'Recall: {re.result().numpy()}')
print(f'Accuracy: {acc.result().numpy()}')

In [ ]:
class_names = ["No Tumor", "Tumor"]

from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_, yhat_binary)

sns.heatmap(conf, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()